# Plant Disease Classification

Notebook by Aditya Tapshalkar

Kaggle dataset: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

---

## Abstract

## Introduction


In [9]:
%env KAGGLEHUB_CACHE=/mnt/devbox/kaggle

import os

import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

torch.manual_seed(42)
print("Torch version:", torch.__version__)
print("Current device:", "cuda" if torch.cuda.is_available() else "cpu")

env: KAGGLEHUB_CACHE=/mnt/devbox/kaggle
Torch version: 2.5.1
Current device: cuda


In [10]:
# Downloading dataset
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
print("Path to dataset files:", path)

Path to dataset files: /mnt/devbox/kaggle/datasets/vipoooool/new-plant-diseases-dataset/versions/2


In [11]:
classes = os.listdir(
    os.path.join(path, "New Plant Diseases Dataset(Augmented)", "New Plant Diseases Dataset(Augmented)", "train")
)

classes

['Soybean___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Tomato___Target_Spot',
 'Grape___Esca_(Black_Measles)',
 'Pepper,_bell___Bacterial_spot',
 'Grape___healthy',
 'Peach___healthy',
 'Tomato___Leaf_Mold',
 'Squash___Powdery_mildew',
 'Raspberry___healthy',
 'Apple___healthy',
 'Apple___Apple_scab',
 'Potato___healthy',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Corn_(maize)___Common_rust_',
 'Tomato___healthy',
 'Blueberry___healthy',
 'Tomato___Late_blight',
 'Tomato___Septoria_leaf_spot',
 'Peach___Bacterial_spot',
 'Tomato___Early_blight',
 'Pepper,_bell___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Bacterial_spot',
 'Cherry_(including_sour)___healthy',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Apple___Cedar_apple_rust',
 'Grape___Black_rot',
 'Strawberry___Leaf_scorch',
 'Grape___Leaf_blight_(Isariop

In [12]:
train_class_list = []
for train_class in classes:
    train_class_list.append({
        'class': train_class,
        'files': os.listdir(os.path.join(
            path, "New Plant Diseases Dataset(Augmented)", "New Plant Diseases Dataset(Augmented)", "train",
            str(train_class)))
    })

In [13]:
df_train = pd.DataFrame(train_class_list)
df_train = df_train.explode('files', ignore_index=True).rename(columns={'files': 'filepath'})
df_train.groupby('class', sort='desc').count()

,filepath
class,
Apple___Apple_scab,2016
Apple___Black_rot,1987
Apple___Cedar_apple_rust,1760
Apple___healthy,2008
Blueberry___healthy,1816
Cherry_(including_sour)___Powdery_mildew,1683
Cherry_(including_sour)___healthy,1826
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot,1642
Corn_(maize)___Common_rust_,1907


In [14]:
val_class_list = []
for val_class in classes:
    val_class_list.append({
        'class': val_class,
        'files': os.listdir(os.path.join(
            path, "New Plant Diseases Dataset(Augmented)", "New Plant Diseases Dataset(Augmented)", "valid",
            str(val_class)))
    })

In [15]:
df_val = pd.DataFrame(val_class_list)
df_val = df_val.explode('files', ignore_index=True).rename(columns={'files': 'filepath'})
df_val.groupby('class', sort='desc').count()

,filepath
class,
Apple___Apple_scab,504
Apple___Black_rot,497
Apple___Cedar_apple_rust,440
Apple___healthy,502
Blueberry___healthy,454
Cherry_(including_sour)___Powdery_mildew,421
Cherry_(including_sour)___healthy,456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot,410
Corn_(maize)___Common_rust_,477


In [ ]:
test_class_list = []
for test_class in classes:
    test_class_list.append({
        'class': test_class,
        'files': os.listdir(os.path.join(
            path, "test",
            str(test_class)))
    })

In [ ]:
df_test = pd.DataFrame(val_class_list)
df_test = df_test.explode('files', ignore_index=True).rename(columns={'files': 'filepath'})
df_test.groupby('class', sort='desc').count()